## Niger Little-Poole
### nl2418
### HW10


In [1]:
import math
import random

<B>1.</B> Control Variates: Consider a GI/GI/1 FIFO queue in which interarrival times {Tn} are
iid distributed as Unif(2, 6), and service times {Sn} are iid distributed as P(S ≤ x) =
$1−e^{−√x}, x ≥ 0.$ ( Thus S has a Weibull distribution: Recall that you can generate it via
generating K exponential at rate 1, and setting $S = K^2$
.) Our objective is to estimate
E(X) where
$$ X =\frac{1}{100} \sum^{100}_{i=1} D_i $$

the average delay over the first 100 customers. Here we assume D0 = 0 and use Dn+1 =
(Dn + Sn − Tn)
+. We note that each copy of X requires 100 copies of S, (S0, . . . , S99),
and 100 copies of T, (T0, . . . , T99).

<b>(a)</b> Generate n = 10, 000 iid copies of X and construct a 95% confidence interval for
estimating E(X) using the so-called raw (standard) estimate
$$ \frac{1}{n} \sum^{n}_{i=1} X_i $$

In [28]:
def service_time():
    u = random.random()
    return math.pow(-1*math.log(u),2)
def interarrival_time():
    u = random.random()
    return 4*u + 2


def delay_time(n):
    d_j = 0
    total = 0.0
    for i in range(n):
        d_j = d_j + service_time() - interarrival_time()
        d_j = d_j if d_j >0 else 0
        total+= d_j
    return total/n

    
def monte_carlo_avg(t,n):
    vals = [ delay_time(t) for x in range(n)]
    mean = sum(vals)/len(vals)
    variance = sum([math.pow( mean - x,2 )for x in vals ])/(n-1)
    return mean,variance

n = 10000
t = 100
m, v = monte_carlo_avg(t,n)
print "n = %d :" % n,m, math.sqrt(v)*1.96/math.sqrt(n)

n = 10000 : 4.17485111967 0.103113902071


<B>(b)</B> For a control variate, we shall use
$$ Z =  \frac{1}{100} \sum^{99}_{i=0} (S_i - T_i) $$

the average of the very same 100 copies of Si − Ti that are used in the construction
of the copy of X. Hence you will generate iid copies of Y = X + c(Z − E(Z)) for an
appropriate value of c, and then construct a 95% confidence interval for estimating
E(Y ) = E(X) using estimate (n = 10, 000 as before)
$$  \frac{1}{n} \sum^{n}_{i=1} Y_i $$

The following explains how you will do this:
First compute $E(Z)$ and $σ^2_Z = Var(Z)$. Then estimate c∗the optimal value of c,

$$ c∗ = − \frac {Cov(X, Z)} {σ^2_Z}$$


In [22]:
def delay_time_cv(n):
    d_j = 0
    total = 0.0
    control = 0.0
    for i in range(n):
        s = service_time()
        t =  interarrival_time()
        d_j = d_j +  s -t
        d_j = d_j if d_j >0 else 0
        total+= d_j
        control += s-t
    return total/n, control/n

    
def monte_carlo_c_star(t,n):
    vals = []
    variates = []
    for i in range(n):
        val, cv = delay_time_cv(t)
        vals.append(val)
        variates.append(cv)
        
    mean = sum(vals)/len(vals)
    mean_cv = sum(variates)/n
    
    variance = sum([math.pow( mean - x,2 )for x in vals ])/(n-1)
    variance_cv =  sum([math.pow( mean - x,2 )for x in variates ])/(n-1)
        
    c_star = 0    
    for i in range(n):
        c_star += (vals[i] - mean) * (variates[i] - mean_cv)
    c_star = -1* c_star / (n-1) / variance_cv
    return c_star, mean_cv

n = 5000
t = 100
C, ez= monte_carlo_c_star(t,n)
print "n = %d :" % n, C,ez

n = 5000 : -0.0437009236641 -1.9965465863


Finally use these to construct a 95% confidence interval for estimating $ E(Y ) = E(X)$

In [25]:
def delay_time_variates(n):
    d_j = 0
    total = 0.0
    control = 0.0
    for i in range(n):
        s = service_time()
        t =  interarrival_time()
        d_j = d_j +  s -t
        d_j = d_j if d_j >0 else 0
        total+= d_j
        control += s-t
    x  = total/n
    z = control/n
    return x + C*(z - ez)

def monte_carlo_variates(t,n):
    vals = [ delay_time_variates(t) for x in range(n)]
    mean = sum(vals)/len(vals)
    variance = sum([math.pow( mean - x,2 )for x in vals ])/(n-1)
    return mean,variance

n = 10000
t = 100
m, v = monte_carlo_variates(t,n)
print "n = %d :" % n,m, math.sqrt(v)*1.96/math.sqrt(n)

n = 10000 : 4.044800681 0.09257765676
